In [1]:
import subprocess
from pydub import AudioSegment
import math
import os

def extract_audio_from_video(video_path,audio_path):
    command=["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command,check=True)
    return audio_path

def split_audio_into_chunks(audio_path,chunk_size,chunks_folder):
    audio = AudioSegment.from_mp3(audio_path)
    chunks = math.ceil(len(audio)/chunk_size)
    chunk_size = chunk_size*60*1000
    if not os.path.exists(chunks_folder):
        os.makedirs(chunks_folder)
    for i in range(chunks):
        chunk = audio[i*chunk_size:(i+1)*chunk_size]
        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")

extract_audio_from_video("files/test.mp4","files/audio.mp3")
split_audio_into_chunks("files/audio.mp3",10,"files/chunks")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --

KeyboardInterrupt: 

In [ ]:
from pydub import AudioSegment

track = AudioSegment.from_mp3("files/audio.mp3")
track.duration_seconds




2529.674739229025

In [10]:
five_minutes = 5 * 60 * 1000

first_five_minutes = track[:five_minutes]

first_five_minutes.export("files/first_five.mp3", format="mp3")

<_io.BufferedRandom name='files/first_five.mp3'>

In [13]:
import math
ten_minutes = 10 * 60 * 1000
chunks = math.ceil(len(track)/ten_minutes)
for i in range(chunks):
    chunk = track[i*ten_minutes:(i+1)*ten_minutes]
    chunk.export(f"files/chunks/chunk_{i}.mp3", format="mp3")

In [4]:
from openai import OpenAI          # openai‑python ≥ 1.0.0
import glob, os

client = OpenAI()                  # OPENAI_API_KEY 환경변수 필요

def transcribe_chunks(chunk_folder: str, destination_folder: str) -> str:
    # 1) mp3 파일 목록을 정렬(숫자순)해서 가져오기 ------------------------------
    files = sorted(
        glob.glob(f"{chunk_folder}/*.mp3"),
        key=lambda p: int(os.path.splitext(os.path.basename(p))[0].split('_')[-1])
    )

    # 2) 각 chunk 전사 --------------------------------------------------------
    transcript_parts = []
    for path in files:
        with open(path, "rb") as f:
            text = client.audio.transcriptions.create(
                model="whisper-1",
                file=f,
                language="ko",        # 한국어
                response_format="text"  # 순수 문자열
            )
            transcript_parts.append(text)  # 그대로 문자열이므로 바로 추가

    final_text = "\n".join(transcript_parts)

    # 3) 출력 폴더 준비 후 저장 -----------------------------------------------
    os.makedirs(destination_folder, exist_ok=True)
    out_path = os.path.join(destination_folder, "transcript.txt")
    with open(out_path, "w", encoding="utf-8") as out:
        out.write(final_text)

    return final_text


# 실행 예시
transcribe_chunks("./files/chunks", "./files/transcripts")


'여러분 안녕하세요 노태우입니다. 입주가 벌써 지났고요. 더위가 물러난다는 처서가 내일 입니다. 그런데도 아직까지 더위는 기승 을 바라고 있습니다. 여러분들도 다 아시다시피 차가운 겨울 날씨가 심장 환자한테 굉장히 불리하다는 것은 아실 거예요. 이미 잘 알고 계실 텐데 사실은 무더운 여름 날씨도 똑같이 심장 환자한테는 불리합니다. 그래서 오늘 말씀드리고자 하는 것은 심장 환자가 건강하게 여름 을 지내기 위해서 어떠한 것들을 조심해야 되는가 이런 걸 몇 가지 좀 정리해서 말씀드릴까 합니다. 올해가 2022년인데 지금 보시는 도표는 2000년간 지구 온도를 보여주는 그림입니다. 실제 정확한 온도 측정이 됐던 것은 1880년이라고 그러네요. 그런데 이제 그 전의 온도는 사실은 과학자들이 추정했던 온도고요 . 그런데 이 도표 한번 보시게 되면요 . 쭉 변화가 없었어요. 2000년 동안 온도가 별 변화가 없었어요 . 그러다 이제 2000년이 되면서부터 갑자기 기온이 상승하는 모습을 볼 수 있고 현재는 인류 역사상 가장 뜨거워진 지구라고 생각되는 그러한 모습입니다. 지금 보시는 도표는 50년 전을 기준으로 했을 때 50년 전을 기준으로 했을 때 최근 10년 사이에 지구 온도 상승 을 보여주는 그림이에요. 그래서 짙게 색깔이 나오는 부분 또 좀 옅게 색깔이 나오는 부분이 50년 전에 비해가지고 기온이 얼마나 상승되어 있는가를 보는데 색깔이 짙을수록 기온 상승이 심한 겁니다. 제일 보시게 되면 지구의 북쪽이 이제 한 4도 정도가 50년 전에 비해 가지고 온도가 올라갔다고 합니다. 지구의 북반부 특히 북극 지역이 굉장히 온도가 굉장히 치솟은 걸 알 수 있어요. 그 결과 북극 쪽에서는 고음이 어디 갈 데가 없어진 겁니다. 그런데 이 더위가 북극 고음만 위협 하는 건 아닙니다. 사람한테는 얼마나 위험할까 이런 연구를 한 아주 흥미로운 연구가 있었어요. 그것도 어디서 했냐면 지구상에서 가장 더운 지역 쿠웨이트에서 연구 를 한 게 있었어요. 이걸 간략하게 소개해드릴까 합니다. 쿠웨이